In [20]:
import requests
import datetime
import time
import pandas as pd
import numpy as np
import pytrends
import matplotlib.pyplot as plt

In [25]:
from pytrends.request import TrendReq

## UTC TIME
pytrends = TrendReq(hl='en-US', tz=0, backoff_factor=0.1)

# kw_list=["Bitcoin crashing", "Test"]
# data = pytrends.get_historical_interest(kw_list, year_start=2018, month_start=1, day_start=1, hour_start=0, year_end=2018, month_end=1, day_end=10, hour_end=0, cat=0, geo='', gprop='', sleep=0)

In [15]:
# data.to_csv("../data/raw/{0}.csv".format("is bitcoin crashing"))

In [26]:
kw_list=["how to buy bitcoin", "bitcoin crash", "where to buy bitcoin", "sell bitcoin", "bitcoin price"]

for kw in kw_list:
    for year in range(2015, 2022): 
        start_month = 7 if year == 2015 else 1
        end_month = 1 if year == 2021 else 12
        data = pytrends.get_historical_interest([kw], year_start=year, month_start=start_month, day_start=1, hour_start=0, year_end=(year+1), month_end=end_month, day_end=31, hour_end=0, cat=0, geo='', gprop='')
        data.to_csv("../data/raw/{0}{1}.csv".format(kw, str(year)))

HTTPSConnectionPool(host='trends.google.com', port=443): Read timed out. (read timeout=5)
HTTPSConnectionPool(host='trends.google.com', port=443): Read timed out. (read timeout=5)
HTTPSConnectionPool(host='trends.google.com', port=443): Read timed out. (read timeout=5)
HTTPSConnectionPool(host='trends.google.com', port=443): Read timed out. (read timeout=5)
HTTPSConnectionPool(host='trends.google.com', port=443): Read timed out. (read timeout=5)
HTTPSConnectionPool(host='trends.google.com', port=443): Read timed out. (read timeout=5)
HTTPSConnectionPool(host='trends.google.com', port=443): Read timed out. (read timeout=5)
HTTPSConnectionPool(host='trends.google.com', port=443): Read timed out. (read timeout=5)
HTTPSConnectionPool(host='trends.google.com', port=443): Read timed out. (read timeout=5)
HTTPSConnectionPool(host='trends.google.com', port=443): Read timed out. (read timeout=5)
HTTPSConnectionPool(host='trends.google.com', port=443): Read timed out. (read timeout=5)
HTTPSConne

HTTPSConnectionPool(host='trends.google.com', port=443): Read timed out. (read timeout=5)
HTTPSConnectionPool(host='trends.google.com', port=443): Read timed out. (read timeout=5)
HTTPSConnectionPool(host='trends.google.com', port=443): Read timed out. (read timeout=5)
HTTPSConnectionPool(host='trends.google.com', port=443): Read timed out. (read timeout=5)
HTTPSConnectionPool(host='trends.google.com', port=443): Read timed out. (read timeout=5)
HTTPSConnectionPool(host='trends.google.com', port=443): Read timed out. (read timeout=5)
HTTPSConnectionPool(host='trends.google.com', port=443): Read timed out. (read timeout=5)
HTTPSConnectionPool(host='trends.google.com', port=443): Read timed out. (read timeout=5)
HTTPSConnectionPool(host='trends.google.com', port=443): Read timed out. (read timeout=5)
HTTPSConnectionPool(host='trends.google.com', port=443): Read timed out. (read timeout=5)
HTTPSConnectionPool(host='trends.google.com', port=443): Read timed out. (read timeout=5)
HTTPSConne

HTTPSConnectionPool(host='trends.google.com', port=443): Read timed out. (read timeout=5)
HTTPSConnectionPool(host='trends.google.com', port=443): Read timed out. (read timeout=5)
HTTPSConnectionPool(host='trends.google.com', port=443): Read timed out. (read timeout=5)
HTTPSConnectionPool(host='trends.google.com', port=443): Read timed out. (read timeout=5)
HTTPSConnectionPool(host='trends.google.com', port=443): Read timed out. (read timeout=5)
HTTPSConnectionPool(host='trends.google.com', port=443): Read timed out. (read timeout=5)
HTTPSConnectionPool(host='trends.google.com', port=443): Read timed out. (read timeout=5)
HTTPSConnectionPool(host='trends.google.com', port=443): Read timed out. (read timeout=5)
HTTPSConnectionPool(host='trends.google.com', port=443): Read timed out. (read timeout=5)
HTTPSConnectionPool(host='trends.google.com', port=443): Read timed out. (read timeout=5)
HTTPSConnectionPool(host='trends.google.com', port=443): Read timed out. (read timeout=5)
HTTPSConne

HTTPSConnectionPool(host='trends.google.com', port=443): Read timed out. (read timeout=5)
HTTPSConnectionPool(host='trends.google.com', port=443): Read timed out. (read timeout=5)
HTTPSConnectionPool(host='trends.google.com', port=443): Read timed out. (read timeout=5)
HTTPSConnectionPool(host='trends.google.com', port=443): Read timed out. (read timeout=5)
HTTPSConnectionPool(host='trends.google.com', port=443): Read timed out. (read timeout=5)
HTTPSConnectionPool(host='trends.google.com', port=443): Read timed out. (read timeout=5)
HTTPSConnectionPool(host='trends.google.com', port=443): Read timed out. (read timeout=5)
HTTPSConnectionPool(host='trends.google.com', port=443): Read timed out. (read timeout=5)
HTTPSConnectionPool(host='trends.google.com', port=443): Read timed out. (read timeout=5)
HTTPSConnectionPool(host='trends.google.com', port=443): Read timed out. (read timeout=5)
HTTPSConnectionPool(host='trends.google.com', port=443): Read timed out. (read timeout=5)
HTTPSConne

HTTPSConnectionPool(host='trends.google.com', port=443): Read timed out. (read timeout=5)
HTTPSConnectionPool(host='trends.google.com', port=443): Read timed out. (read timeout=5)
HTTPSConnectionPool(host='trends.google.com', port=443): Read timed out. (read timeout=5)
HTTPSConnectionPool(host='trends.google.com', port=443): Read timed out. (read timeout=5)
HTTPSConnectionPool(host='trends.google.com', port=443): Read timed out. (read timeout=5)
HTTPSConnectionPool(host='trends.google.com', port=443): Read timed out. (read timeout=5)
HTTPSConnectionPool(host='trends.google.com', port=443): Read timed out. (read timeout=5)
HTTPSConnectionPool(host='trends.google.com', port=443): Read timed out. (read timeout=5)
HTTPSConnectionPool(host='trends.google.com', port=443): Read timed out. (read timeout=5)
The request failed: Google returned a response with code 500.
The request failed: Google returned a response with code 400.
The request failed: Google returned a response with code 400.
The 

...

The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The requ

The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The requ

The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The requ

The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The requ

The request failed: Google returned a response with code 400.
The request failed: Google returned a response with code 400.
The request failed: Google returned a response with code 400.
The request failed: Google returned a response with code 400.
The request failed: Google returned a response with code 400.
The request failed: Google returned a response with code 400.
The request failed: Google returned a response with code 400.
The request failed: Google returned a response with code 400.
The request failed: Google returned a response with code 400.
The request failed: Google returned a response with code 400.
The request failed: Google returned a response with code 400.
The request failed: Google returned a response with code 400.
The request failed: Google returned a response with code 400.
The request failed: Google returned a response with code 400.
The request failed: Google returned a response with code 400.
The request failed: Google returned a response with code 400.
The requ

In [61]:
kw_list=["how to buy bitcoin", "bitcoin crash", "where to buy bitcoin", "sell bitcoin", "bitcoin price"]
pytrends.interest_over_time(kw_list=kw_list, timeframe='2016-12-14 2017-01-25')

0